In [1]:
# mpc_core.py
# --------------------
# Módulo central para MPC: AuthShare, interfaz de red y Reshare.
# Diseñado para ejecutarse en Sage (usa GF, elementos exactos).
#

# --------------------

from sage.all import GF
from typing import List, Tuple, Dict, Any

# -----------------------------
# Clase AuthShare
# -----------------------------
class AuthShare:
    """
    Representa una compartición autenticada <x>:
      - F      : campo (GF(p))
      - delta  : por si se usa en extensiones (no usado intensamente ahora)
      - shares : lista de shares locales (cada jugador i tiene shares[i])
      - macs   : lista de macs locales (cada jugador i tiene macs[i])
    Métodos:
      - value() : reconstruye el secreto (suma de shares)
      - mac_sum(): suma de las macs (debe ser alpha * value())
    """
    def __init__(self, F, delta, shares: List[Any], macs: List[Any]):
        self.F = F
        self.delta = F(delta)
        self.shares = [F(s) for s in shares]
        self.macs = [F(m) for m in macs]

    def value(self):
        # retorna el secreto (suma de shares) como elemento de F
        return sum(self.shares)

    def mac_sum(self):
        return sum(self.macs)

    def n_players(self):
        return len(self.shares)

# -----------------------------
# Función helper: generar AuthShare consistente (preprocessing simulado)
# -----------------------------
def make_rand_auth(F, v, alpha, n):
    """
    Construye una AuthShare aleatoria y consistente:
      - v: valor secreto (elemento de F o convertible)
      - alpha: clave global
      - n: número de players
    Útil para testing local / demos.
    """
    shares = [F.random_element() for _ in range(n-1)]
    shares.append(F(v - sum(shares)))
    delta = F(0)
    mac_target = alpha * (v + delta)
    macs = [F.random_element() for _ in range(n-1)]
    macs.append(mac_target - sum(macs))
    return AuthShare(F, delta, shares, macs)

# -----------------------------
# Interfaz de red (abstracta)
# -----------------------------
class NetworkInterface:
    """
    Interfaz requerida por el protocolo reshare.
    Debes implementar estas funciones si conectas a una red real.

    Requisitos mínimos:
      - start_round(): iniciar un nuevo round de comunicación
      - broadcast(sender_id: int, value): enviar un valor (publicar)
      - collect(): devuelve lista de tuplas (sender_id, value) recolectadas en este round
      - end_round(): opcional para limpieza
    """
    def start_round(self):
        raise NotImplementedError
    def broadcast(self, sender_id: int, value):
        raise NotImplementedError
    def collect(self) -> List[Tuple[int, Any]]:
        raise NotImplementedError
    def end_round(self):
        raise NotImplementedError

# -----------------------------
# Implementación de red simulada (para pruebas locales)
# -----------------------------
class SimNetwork(NetworkInterface):
    """
    Simula a n players. Cada broadcast registra (sender_id, value).
    collect() devuelve todos los valores enviados en la ronda actual.
    """
    def __init__(self, n):
        self.n = n
        self._buffer = []
    def start_round(self):
        self._buffer = []
    def broadcast(self, sender_id: int, value):
        # guardamos un par (sender_id, value)
        self._buffer.append((sender_id, value))
    def collect(self):
        # en simulación devolvemos lo que se envió; en red real, bloqueas hasta recibir n mensajes
        return list(self._buffer)
    def end_round(self):
        pass

# -----------------------------
# RESHARE: versión práctica (última mac-share ajustada)
# -----------------------------
def reshare_practical(F, X_auth: AuthShare, R_auth: AuthShare, alpha, network: NetworkInterface) -> Tuple[AuthShare, Dict]:
    """
    Reshare práctico: abre eps = X - R (broadcast), construye nuevos shares r_i + eps,
    y ajusta la última mac-share con eps * alpha.
    Retorna (new_auth, transcript)
    Transcript es un dict con datos útiles para auditoría.
    """
    n = X_auth.n_players()
    assert n == R_auth.n_players(), "X and R must have same number of players"
    network.start_round()

    # ROUND 1: cada jugador publica d_i = x_i - r_i
    # (en la simulación: llamamos broadcast para cada jugador)
    local_d = [X_auth.shares[i] - R_auth.shares[i] for i in range(n)]
    for i in range(n):
        network.broadcast(i, local_d[i])

    # colectar todos los d_j
    msgs = network.collect()
    # en implementación real: verificar que msgs contenga exactamente n entradas y validarlas
    # reconstructor eps:
    eps = sum([v for (_, v) in msgs])
    eps = F(eps)

    # construir nuevos shares y macs
    new_shares = [R_auth.shares[i] + eps for i in range(n)]
    new_macs = [R_auth.macs[i] for i in range(n)]
    new_macs[-1] = new_macs[-1] + (eps * alpha)  # convención práctica

    new_auth = AuthShare(F, R_auth.delta, new_shares, new_macs)

    transcript = {
        "round_msgs": [(int(sid), int(v)) for (sid, v) in msgs],
        "eps": int(eps),
        "new_shares": [int(s) for s in new_shares],
        "new_macs": [int(m) for m in new_macs],
    }

    network.end_round()
    return new_auth, transcript

# -----------------------------
# RESHARE: versión distribuida (alpha_shares)
# -----------------------------
def reshare_distributed(F, X_auth: AuthShare, R_auth: AuthShare, alpha_shares: List[Any], network: NetworkInterface) -> Tuple[AuthShare, Dict]:
    """
    Variante distribuida donde la corrección eps*alpha se reparte según alpha_shares (lista que suma alpha).
    """
    n = X_auth.n_players()
    assert n == R_auth.n_players()
    assert len(alpha_shares) == n

    network.start_round()
    local_d = [X_auth.shares[i] - R_auth.shares[i] for i in range(n)]
    for i in range(n):
        network.broadcast(i, local_d[i])

    msgs = network.collect()
    eps = sum([v for (_, v) in msgs])
    eps = F(eps)

    new_shares = [R_auth.shares[i] + eps for i in range(n)]
    new_macs = [R_auth.macs[i] + (eps * F(alpha_shares[i])) for i in range(n)]

    new_auth = AuthShare(F, R_auth.delta, new_shares, new_macs)

    transcript = {
        "round_msgs": [(int(sid), int(v)) for (sid, v) in msgs],
        "eps": int(eps),
        "new_shares": [int(s) for s in new_shares],
        "new_macs": [int(m) for m in new_macs],
    }

    network.end_round()
    return new_auth, transcript

# -----------------------------
# Verificación simple de una AuthShare
# -----------------------------
def verify_authshare_consistency(auth: AuthShare, alpha) -> bool:
    """
    Comprueba que sum(macs) == alpha * sum(shares).
    """
    return int(auth.mac_sum()) == int(alpha * auth.value())

# -----------------------------
# END mpc_core.py
# -----------------------------


In [12]:
def reshare_practical(F, X_auth, R_auth, alpha, network, preserve_secret=True):
    """
    RESHARE práctico.
    - Si preserve_secret==True: construye new_auth cuya reconstrucción (value()) == x (original).
    - Si preserve_secret==False: construye new_auth con delta aleatorio (value() == x + delta_new).
    Retorna: (AuthShare, transcript)
    Transcript contiene 'eps' y mensajes broadcast.
    """
    n = X_auth.n_players()
    assert n == R_auth.n_players()

    network.start_round()

    # 1) Cada jugador publica d_i = x_i - r_i
    d_i = [X_auth.shares[i] - R_auth.shares[i] for i in range(n)]
    for i in range(n):
        network.broadcast(i, d_i[i])

    # 2) Recolectar y reconstruir eps
    msgs = network.collect()                 # espera la lista de (sender, value)
    eps = sum(v for (_, v) in msgs)
    eps = F(eps)

    # 3) Reconstruimos x = r + eps
    r = R_auth.value()
    x = r + eps

    # 4) Construcción de nuevo AuthShare
    if preserve_secret:
        # Queremos que new.value() == x EXACTAMENTE (no introducir delta).
        delta_new = F(0)
        target = x
    else:
        # Introducimos un delta nuevo; new.value() == x + delta_new
        delta_new = F.random_element()
        target = x + delta_new

    # Generar n-1 shares aleatorios y fijar el último para que sumen 'target'
    new_shares = [F.random_element() for _ in range(n - 1)]
    last_share = target - sum(new_shares)
    new_shares.append(last_share)

    # Generar MACs consistentes: sum(macs) == alpha * target
    mac_target = alpha * target
    new_macs = [F.random_element() for _ in range(n - 1)]
    last_mac = mac_target - sum(new_macs)
    new_macs.append(last_mac)

    network.end_round()

    new_auth = AuthShare(F, delta_new, new_shares, new_macs)
    transcript = {
        "round_msgs": [(int(sid), int(v)) for (sid, v) in msgs],
        "eps": int(eps),
        "target_reconstructed": int(target)
    }
    return new_auth, transcript



In [13]:
# Parámetros
p = 2**61 - 1
F = GF(p)
n = 3
alpha = F.random_element()

# Generar X y R consistentes (preprocessing simulado)
x_val = F.random_element()
r_val = F.random_element()
X = make_rand_auth(F, x_val, alpha, n)
R = make_rand_auth(F, r_val, alpha, n)

net = SimNetwork(n)

# Opción A: preservar secreto
new_X_preserve, tr_preserve = reshare_practical(F, X, R, alpha, net, preserve_secret=True)
print("preserve -> secret orig:", int(X.value()))
print("preserve -> secret new :", int(new_X_preserve.value()))
print("preserve -> mac ok?    :", verify_authshare_consistency(new_X_preserve, alpha))

# Opción B: delta nuevo
new_X_delta, tr_delta = reshare_practical(F, X, R, alpha, net, preserve_secret=False)
print("delta_new -> secret orig:", int(X.value()))
print("delta_new -> secret new :", int(new_X_delta.value()))
print("delta_new -> mac ok?    :", verify_authshare_consistency(new_X_delta, alpha))


preserve -> secret orig: 2028147084423107005
preserve -> secret new : 2028147084423107005
preserve -> mac ok?    : True
delta_new -> secret orig: 2028147084423107005
delta_new -> secret new : 81335783294089780
delta_new -> mac ok?    : True
